In [44]:
using StableSpectralElements, OrdinaryDiffEq, LinearAlgebra, StartUpDG
using Plots, Plots.PlotMeasures, Printf, TimerOutputs, BenchmarkTools, PrettyTables

In [29]:
a = (1.0,1.0,1.0)  # advection velocity
L = 1.0  # domain length
T = 1.0  # end time

conservation_law = LinearAdvectionEquation(a)
exact_solution = InitialDataSine(1.0,(2π/L, 2π/L, 2π/L));

In [65]:
M = 2
p = 1

reference_approximation_tensor = ReferenceApproximation(
    ModalTensor(p), Tet(), mapping_degree=1, 
    volume_quadrature_rule=(LGQuadrature(p),
        LGQuadrature(p), GaussQuadrature(p,1,0)), 
    facet_quadrature_rule=(LGQuadrature(p), GaussQuadrature(p,1,0)))

reference_approximation_multi = ReferenceApproximation(
    ModalMulti(p), Tet(), mapping_degree=1, 
    volume_quadrature_rule=(LGQuadrature(p),
        LGQuadrature(p), GaussQuadrature(p,1,0)), 
    facet_quadrature_rule=(LGQuadrature(p), GaussQuadrature(p,1,0)))

form = StandardForm(mapping_form=StandardMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux())

uniform_mesh = uniform_periodic_mesh(
            reference_approximation.reference_element, 
            Tuple((0.0,L) for m in 1:3), Tuple(M for m in 1:3),
            collapsed_orientation=true, tol=1.0e-10)

mesh = warp_mesh(uniform_mesh, reference_approximation, 
    ChanWarping(1.0/16.0,Tuple(L for m in 1:3)))

spatial_discretization_tensor = SpatialDiscretization(mesh, 
    reference_approximation_tensor, ExactMetrics())

spatial_discretization_multi = SpatialDiscretization(mesh, 
    reference_approximation_multi, ExactMetrics());

In [66]:
C_t = 0.05
h = L/M
dt = C_t*h/(norm(a)*p^2)

ode_problem_tensor = semidiscretize(conservation_law, spatial_discretization_tensor,
    exact_solution, form, (0.0, T), PhysicalOperator());
ode_problem_multi = semidiscretize(conservation_law, spatial_discretization_multi,
    exact_solution, form, (0.0, T), PhysicalOperator());

In [67]:
println(round.(Matrix(ode_problem_tensor.p.operators.VOL[1][1]) 
        .- Matrix(ode_problem_multi.p.operators.VOL[1][1]); digits=3))

[0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 0.0; -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0; 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; -0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0]


In [68]:
println(round.(Matrix(ode_problem_tensor.p.operators.FAC[1]) 
        .- Matrix(ode_problem_multi.p.operators.FAC[1]); digits=3))

[0.0 -0.0 0.0 0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 0.0 0.0; -0.0 -0.0 -0.0 -0.0 5.818 -2.376 0.267 -3.71 0.0 0.0 0.0 -0.0 0.0 0.0 -0.0 0.0; -0.0 0.0 -0.0 0.0 1.773 3.095 -6.077 1.208 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 0.0; -0.0 -0.0 0.0 0.0 -3.399 -0.817 3.399 0.817 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0]


In [75]:
println(round.(Matrix(reference_approximation_tensor.R * reference_approximation_tensor.V) 
        .- Matrix(reference_approximation_tensor.Vf); digits=3))

[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [82]:
println(round.(Matrix(reference_approximation_multi.R * reference_approximation_multi.V) 
        .- Matrix(reference_approximation_multi.Vf); digits=3))

[0.0 -0.0 0.0 -0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 -0.0 -0.0 0.0; -0.0 0.0 -0.0 0.0; -0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0; 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0; -0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [83]:
reference_approximation_multi.reference_element.rf .- reference_approximation_tensor.reference_element.rf

16-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
 -0.9756630355021699
 -0.4099776105529319
  0.9756630355021702
  0.4099776105529319
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0